In [28]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Read data from Excel
file_path = "study_plan.xlsx"
activities_df = pd.read_excel(file_path, sheet_name="Activities")
availability_df = pd.read_excel(file_path, sheet_name="Week Availability")
semester_info_df = pd.read_excel(file_path, sheet_name="Semester Info")

# Extract semester information
num_weeks = int(semester_info_df["Num Weeks"][0])
plan_weeks = list(availability_df["Week"])

# Extract activities, required hours, due dates, and earliest start weeks
activities = activities_df["Activity"].tolist()
required_hours = dict(zip(activities_df["Activity"], activities_df["Required Hours"]))
due_dates = dict(zip(activities_df["Activity"], zip(activities_df["Due Week"], activities_df["Due Day"])))
earliest_start = dict(zip(activities_df["Activity"], activities_df["Earliest Start Week"]))

# Days of the week
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Convert Week Availability to a dictionary
availability = {}
for week_index in range(num_weeks):
    for day in days:
        if (availability_df.loc[week_index, "Week"], day) not in availability:
            availability[(availability_df.loc[week_index, "Week"], day)] = availability_df.loc[week_index, day]

# Initialize the model
model = gp.Model("Study_Plan")

# Decision variables: x[i, week, day] is the study hours for activity i in week 'week' on day 'day'
x = model.addVars(activities, plan_weeks, days, name="study_hours", vtype=GRB.CONTINUOUS)

# Auxiliary variables to capture min/max workload ratios
min_ratio = model.addVars(plan_weeks, days, name="min_ratio", vtype=GRB.CONTINUOUS)
max_ratio = model.addVars(plan_weeks, days, name="max_ratio", vtype=GRB.CONTINUOUS)

# Define weights for the multi-objective function
weight_total_hours = 0.8  # Higher weight for minimizing total study hours
weight_workload_deviation = 0.2  # Lower weight for minimizing workload deviation
max_study_hours = sum(list(availability.values()))

# Objective: Minimize weighted sum of total study hours and workload deviation
model.setObjective(
    weight_total_hours * gp.quicksum(x[i, week, day] for i in activities for week in plan_weeks for day in days) / max_study_hours +
    weight_workload_deviation * gp.quicksum(max_ratio[week, day] - min_ratio[week, day] for week in plan_weeks for day in days),
    GRB.MINIMIZE)

# Constraints for workload ratio per day
for week in plan_weeks:
    for day in days:
        total_study_hours = gp.quicksum(x[i, week, day] for i in activities)
        model.addConstr(min_ratio[week, day] <= total_study_hours / availability[(week, day)], name=f"min_ratio_constraint_{week}_{day}")
        model.addConstr(max_ratio[week, day] >= total_study_hours / availability[(week, day)], name=f"max_ratio_constraint_{week}_{day}")

# Constraint: Study hours per activity must meet the required hours
for i in activities:
    model.addConstr(gp.quicksum(x[i, week, day] for week in plan_weeks for day in days) >= required_hours[i], name=f"required_hours_{i}")

# Constraint: Daily study time limit for each week
for week in plan_weeks:
    for day in days:
        model.addConstr(gp.quicksum(x[i, week, day] for i in activities) <= availability[(week, day)], name=f"daily_limit_week{week}_{day}")

# Constraint: Ensure activities are completed by their due dates
for i, (due_week, due_day) in due_dates.items():
    model.addConstr(gp.quicksum(x[i, week, day] for week in range(plan_weeks[0],due_week+1) for day in days) >= required_hours[i], name=f"due_date_{i}")

# Constraint: Earliest start week for activities 
for i in activities:
    for week in range(plan_weeks[0], earliest_start[i]):
        for day in days:
            model.addConstr(x[i, week, day] == 0, name=f"earliest_start_{i}_{week}_{day}")

# Solve the model
model.optimize()

# Check if a solution is found and export to Excel
if model.status == GRB.OPTIMAL:

    total_hours = gp.quicksum(x[i, week, day].x for i in activities for week in plan_weeks for day in days)
    print(f"Optimal total hours: {total_hours}")


    # Create a list to store the study plan output
    study_plan_output = []
    
    for week in plan_weeks:
        for day in days:
            for i in activities:
                hours_allocated = x[i, week, day].x
                if hours_allocated > 0:
                    study_plan_output.append([week, day, i, hours_allocated])
    
    # Convert the study plan to a pandas DataFrame
    study_plan_df = pd.DataFrame(study_plan_output, columns=['Week', 'Day', 'Activity', 'Study Hours'])

    # Write the DataFrame to an Excel file
    output_file = "optimized_study_plan.xlsx"
    study_plan_df.to_excel(output_file, index=False)

    print(f"Study plan successfully saved to {output_file}")




else:
    print("No optimal solution found")




Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 885 rows, 2058 columns and 9352 nonzeros
Model fingerprint: 0x97c926ff
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [2e-03, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 752 rows and 749 columns
Presolve time: 0.01s
Presolved: 133 rows, 1309 columns, 3094 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.275000e+01   0.000000e+00      0s
      75    2.4235294e-01   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

      57 PPushes remaining with PInf 0.0000000e+00                 0s
       0 PPushes remaining with PInf 0.

<ipython-input-28-2aef22b77e99>:103: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  study_plan_df.to_excel(output_file, index=False)
